In [40]:
# Importar las librerías necesarias
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import nbformat

In [ ]:
# Transformacion de los datos Municipios y generar dataset
# Cargar el archivo CSV
municipios_df = pd.read_csv('municipios/Municipios_Chip.csv')

# Se crea una nueva columna con el formato "IdEntidad - Municipio"
municipios_df['Entidad_Municipio'] = municipios_df['IdEntidad'].astype(str) + ' - ' + municipios_df['RazónSocial']

# Mostrar el dataframe resultante
municipios = (municipios_df[['Entidad_Municipio']])
municipiosid = (municipios_df[['IdEntidad']])
# print("Municipios", municipios)
# print("IdEntidad", municipiosid)

In [52]:
# Leer la variable municipios y destination_dir desde el archivo data.ipynb
notebook_path = 'data.ipynb'
with open(notebook_path, 'r') as f:
    nb = nbformat.read(f, as_version=4)

# Buscar la celda que contiene las variables municipios y destination_dir
municipios = None
destination_dir = "/Users/cesar/Documents/Importacion/activos"
for cell in nb.cells:
    if cell.cell_type == 'code':
        exec(cell.source)
        if 'municipios_df' in cell.source:
            municipios = municipios_df
        if municipios is not None:
            break

# Configurar el driver de Chrome
driver = webdriver.Chrome()

# Configurar la carpeta de descargas
download_dir = "/Users/cesar/Downloads"  # Ruta de la carpeta de descargas por defecto

# Lista para almacenar los municipios que no pudieron procesarse
municipios_no_procesados = []

# Iterar sobre cada municipio en el dataframe
for index, row in municipios.iterrows():
    municipio = row['Entidad_Municipio']
    expected_filename = f"{municipio}.xls"
    
    # Verificar si el archivo ya existe
    if os.path.exists(os.path.join(destination_dir, expected_filename)):
        print(f"El archivo {expected_filename} ya existe. Saltando descarga.")
        continue
    
    try:
        # Acceder a la página
        driver.get("https://www.chip.gov.co/schip_rt/index.jsf")

        # Esperar a que la página esté completamente cargada
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script('return document.readyState') == 'complete'
        )

        # Navegar a la sección "Consultas"
        consultas_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Consultas"))
        )
        consultas_link.click()

        # Navegar a "Informe al Ciudadano"
        informe_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.LINK_TEXT, "Informe al Ciudadano"))
        )
        informe_link.click()

        # Buscar el ID del municipio
        search_box = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#frm1\\:SelBoxEntidadCiudadano input[id='frm1:SelBoxEntidadCiudadano_input']"))
        )
        search_box.clear()
        search_box.send_keys(municipio)
        search_box.send_keys(Keys.RETURN)

        time.sleep(1)
        # Seleccionar la categoría "FUT_GASTOS_DE_INVERSION" del select con el ID "frm1:SelBoxCategoria"
        categoria_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#frm1\\:SelBoxCategoria"))
        )
        select = Select(categoria_select)
        
        for option in select.options:
            if option.text == "FUT_GASTOS_DE_INVERSION":
                option.click()
                break

        time.sleep(1)
        # Seleccionar el periodo "OCT A DIC - 2009" del select con el ID "frm1:SelBoxPeriodo"
        periodo_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#frm1\\:SelBoxPeriodo"))
        )
        select = Select(periodo_select)
        
        for option in select.options:
            if option.text == "OCT A DIC - 2009":
                option.click()
                break

        time.sleep(1)
        # Seleccionar Formulario "GASTOS_DE_INVERSION" del select con el ID "frm1:SelBoxForma"
        forma_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#frm1\\:SelBoxForma"))
        )
        select = Select(forma_select)
        
        for option in select.options:
            if option.text == "GASTOS_DE_INVERSION":
                option.click()
                break

        time.sleep(1)
        # Hacer clic en el botón "Consultar"
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "frm1:BtnConsular"))
        )
        search_button.click()

        time.sleep(1)

        # Hacer clic en la imagen para descargar el archivo
        download_link = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[title='Descargar a Excel'] img"))
        )
        download_link.click()

        # Extraer el texto del span
        span_text = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "frm1:j_idt152"))
        ).text

        time.sleep(1)

        # Renombrar y mover el archivo descargado
        for filename in os.listdir(download_dir):
            if filename.endswith(".xls"):  # Ajusta la extensión según el tipo de archivo descargado
                source_path = os.path.join(download_dir, filename)
                destination_path = os.path.join(destination_dir, f"{span_text}.xls")
                os.rename(source_path, destination_path)
                break

        time.sleep(1)

    except Exception as e:
        print(f"Error al procesar el municipio {municipio}: {e}")
        print(f"Detalles del error: {e.__class__.__name__}: {e}")
        municipios_no_procesados.append(municipio)
        
# Guardar los municipios que no pudieron procesarse en un archivo CSV
if municipios_no_procesados:
    df_no_procesados = pd.DataFrame(municipios_no_procesados, columns=['Municipio'])
    df_no_procesados.to_csv('municipios_no_procesados.csv', index=False)

# Cerrar el navegador
driver.quit()

Municipios                                       Entidad_Municipio
0                                    89970221 - Coveñas
1                      210005400 - La Unión - Antioquia
2                           210013300 - Hatillo de Loba
3                                  210013600 - Rioviejo
4                                    210015500 - Oicatá
...                                                 ...
1108  923272864 - Fundación San Juan de Dios de Sant...
1109  923272915 - Infraestructura Asset Management C...
1110                          923272927 - Barrancominas
1111             923273537 - Rotorr-Motor de Innovación
1112            923273541 - Impacto infraestructura SAS

[1113 rows x 1 columns]
IdEntidad       IdEntidad
0      89970221
1     210005400
2     210013300
3     210013600
4     210015500
...         ...
1108  923272864
1109  923272915
1110  923272927
1111  923273537
1112  923273541

[1113 rows x 1 columns]
El archivo 89970221 - Coveñas.xls ya existe. Saltando descarga.
E